In [1]:
import plotly.offline as plt
from plotly import tools
import plotly.graph_objs as go
from plotly.graph_objs import Layout
from plotly.graph_objs.graph_objs import Scatter, Data
from ipywidgets import *
import ipywidgets as widgets
import math

plt.init_notebook_mode(connected=True)

In [5]:
class MapToD(object):
    """Class for chowing 2D map for N and H chemical shifts per steps"""
    def __init__(self, titration):
        """Defines ettributs
        argument 'titration' is a dicitionnary of 2 dimensions list for each key (should be a titration object from shiftome after developement)"""
        self.titration=titration
        self.selector=widgets.SelectMultiple(options=[key for key in self.titration],
                                             rows=min(9,len(self.titration)),
                                             description='Residue ',
                                             disabled=False) # multiple selector widget for selecting one or more residues
        self.split=widgets.ToggleButton(value=False,
                                        description='Split map',
                                        disabled=False,
                                        button_style='success',
                                        tooltip='Description',
                                        icon='check')
    def nonSplitted(self, residues):
        data=[]
        for res in residues:
            trace=go.Scatter(x=self.titration[res][0], 
                                y=self.titration[res][1],
                                mode='markers', 
                                name='Residue '+str(res), 
                                text=["Step "+str(i+1) for i in range(len(self.titration[res][0]))],
                                marker={'color':[index+1 for index in range(len(self.titration[res][0]))], # sets a color scale regarding step number
                                        'colorbar':{'title':'Steps', 'titlefont':{'size':16}},
                                        'colorscale':'Bluered',
                                        'size':10,
                                        'line':{'width':2,
                                                'colorscale':'Viridis'}})
            data.append(trace)
        fig=go.Figure(data=data)
        return fig
    def splitted(self, residues):
        data=[]
        rows=math.ceil(math.sqrt(len(residues))) # defines appropriate number of rows 
        cols=round(math.sqrt(len(residues))) # defines appropraite number of columns
        fig = tools.make_subplots(rows=rows, cols=cols, print_grid=False)
        colNumber=0
        rowNumber=1
        for res in residues:
            ##by-columns-then-by-lines method for plot positionning
            if colNumber < cols:
                colNumber+=1
            elif rowNumber < rows:
                colNumber =1
                rowNumber +=1
            ##
            trace=go.Scatter(x=self.titration[res][0], 
                                y=self.titration[res][1],
                                mode='markers', 
                                name='Residue '+str(res), 
                                text=["Step "+str(i+1) for i in range(len(self.titration[res][0]))],
                                marker={'color':[index+1 for index in range(len(self.titration[res][0]))], # sets a color scale regarding step number
                                        'colorbar':{'title':'Steps', 'titlefont':{'size':16}},
                                        'colorscale':'Bluered',
                                        'size':20-(rows*cols),
                                        'line':{'width':2,
                                                'color':'rgb(0, 0, 0)'}})
            fig.append_trace(trace, rowNumber,colNumber)
        return fig
    
    def scatterPlot(self, residues, splitted):
        """Method that traces scatter plot of N and H chemical shift per step for one or more residues.
        Creates a scatter plot for each residue given in a sublpot"""
        if residues ==():
            """
            residues=tuple([residue for residue in self.titration])
            print(residues)
            fig=self.nonSplitted(residues)
            """
            return
        elif splitted==False: #no residues selected
            fig=self.nonSplitted(residues)
        elif splitted==True:
            fig=self.splitted(residues)
           
        fig['layout'].update(title='H and N chemical shifts of residue '+", ".join(map(str, residues)),
                            showlegend=False,
                            annotations=[{
                                'x':1, #x position of annotation
                                'y':5,
                                'xref':'x', #plot number
                                'yref':'y',
                                'text':'max=5',
                                'showarrow':False,
                                'font':{
                                    'family':'Courier New, monospace',
                                    'size':11,
                                     'color':'#ffffff'
                                },
                                'bordercolor':'rgb(255,255,255)',
                                'borderwidth':1,
                                'borderpad':4,
                                'bgcolor':'rgb(0,0,0)',
                                'opacity':0.7
                            }]*len(residues)+[{'xref': 'paper',
                                               'yref': 'paper',
                                               'x': 0, #x position of x ax legend
                                               'xanchor': 'right',
                                               'y': 1,
                                               'yanchor': 'bottom',
                                               'text': 'N Chemical shift',
                                               'font':{'size':16},
                                               'showarrow': False},
                                              {'xref': 'paper',
                                               'yref': 'paper',
                                               'x': 1,
                                               'xanchor': 'left',
                                               'y': 0,
                                               'yanchor': 'top',
                                               'text': 'H chemical shift',
                                               'font':{'size':16},
                                               'showarrow': False}],
                             width=1000, #width of plotly interface
                             height=800,
                             margin=go.Margin(
                                 l=200,
                                 r=200,
                                 b=100,
                                 t=100,
                                 pad=4))
        if splitted:
            fig['layout']['shapes']=[]
            for index,res in enumerate(residues):
                fig['layout']['annotations'][index]['xref']="x"+str(index+1) #annotation affectation
                fig['layout']['annotations'][index]['yref']="y"+str(index+1)
                fig['layout']['annotations'][index]['text']="Residue " +str(res)
                fig['layout']['annotations'][index]['font']['size']=max(1,11-0.1*len(residues))
                fig['layout']['annotations'][index]['x']=(max(self.titration[res][0])+min(self.titration[res][0]))/2 #annotation: middle plot affectation
                fig['layout']['annotations'][index]['y']=1.2*max(self.titration[res][1])#annotation: top plot affectation
                fig['layout']['xaxis'+str(index+1)].update(showline=True,gridwidth=5) #show x ax
                fig['layout']['yaxis'+str(index+1)].update(showline=True,gridwidth=5)
        else:
            fig['layout']['annotations']=[fig['layout']['annotations'][-1]]
            listColor=['blue', 'red', 'purple', 'black', 'yellow', 'orange', 'grey']
            for index,res in enumerate(residues):
                fig['data'][index]['marker']['line']['color']='black'
                
        plt.iplot(fig)
            
                 
titrationObj={1:[[10.422,11.452452,15.42452,16.4252,16.545,17.45243,17.45278],[1,2,2,5,7,6]] , 
              2:[[14,11,10,13,16,16,17], [4,5,2,3,6,7,8]],
              3:[[10,11,15,11,12,13,14],[1,2,2,5,4,3,6]] , 
              4:[[14,11,10,13,16,16,17], [4,5,2,3,6,7,8]],
              5:[[10,11,15,11,12,13,14],[1,2,2,5,4,3,6]] , 
              6:[[14,11,10,13,16,16,17], [4,5,2,3,6,7,8]],
              8:[[10,11,15,11,12,13,14],[1,2,2,5,4,3,6]] , 
              9:[[14,11,10,13,16,16,17], [4,5,2,3,6,7,8]],
              10:[[10,11,15,11,12,13,14],[1,2,2,5,4,3,6]] , 
              12:[[14,11,10,13,16,16,17], [4,5,2,3,6,7,8]],
              13:[[10,11,15,11,12,13,14],[1,2,2,5,4,3,6]] , 
              14:[[14,11,10,13,16,16,17], [4,5,2,3,6,7,8]],
              15:[[10,11,15,11,12,13,14],[1,2,2,5,4,3,6]] , 
              16:[[14,11,10,13,16,16,17], [4,5,2,3,6,7,8]],
              18:[[10,11,15,11,12,13,14],[1,2,2,5,4,3,6]] , 
              19:[[14,11,10,13,16,16,17], [4,5,2,3,6,7,8]],
              21:[[10,11,15,11,12,13,14],[1,2,2,5,4,3,6]] , 
              22:[[14,11,10,13,16,16,17], [4,5,2,3,6,7,8]],
              23:[[10,11,15,11,12,13,14],[1,2,2,5,4,3,6]] , 
              24:[[14,11,10,13,16,16,17], [4,5,2,3,6,7,8]],
              25:[[10,11,15,11,12,13,14],[1,2,2,5,4,3,6]] , 
              26:[[14,11,10,13,16,16,17], [4,5,2,3,6,7,8]],
              28:[[10,11,15,11,12,13,14],[1,2,2,5,4,3,6]] , 
              29:[[14,11,10,13,16,16,17], [4,5,2,3,6,7,8]],
              31:[[10.422,11.452452,15.42452,16.4252,16.545,17.45243,17.45278],[1,2,2,5,7,6]] , 
              32:[[14,11,10,13,16,16,17], [4,5,2,3,6,7,8]],
              33:[[10,11,15,11,12,13,14],[1,2,2,5,4,3,6]] , 
              34:[[14,11,10,13,16,16,17], [4,5,2,3,6,7,8]],
              35:[[10,11,15,11,12,13,14],[1,2,2,5,4,3,6]] , 
              36:[[14,11,10,13,16,16,17], [4,5,2,3,6,7,8]],
              38:[[10,11,15,11,12,13,14],[1,2,2,5,4,3,6]] , 
              39:[[14,11,10,13,16,16,17], [4,5,2,3,6,7,8]],
              40:[[10,11,15,11,12,13,14],[1,2,2,5,4,3,6]] , 
             112:[[14,11,10,13,16,16,17], [4,5,2,3,6,7,8]],
              113:[[10,11,15,11,12,13,14],[1,2,2,5,4,3,6]] , 
              114:[[14,11,10,13,16,16,17], [4,5,2,3,6,7,8]],
              115:[[10,11,15,11,12,13,14],[1,2,2,5,4,3,6]] , 
              116:[[14,11,10,13,16,16,17], [4,5,2,3,6,7,8]],
              118:[[10,11,15,11,12,13,14],[1,2,2,5,4,3,6]] , 
              119:[[14,11,10,13,16,16,17], [4,5,2,3,6,7,8]],
              121:[[10,11,15,11,12,13,14],[1,2,2,5,4,3,6]] , 
              122:[[14,11,10,13,16,16,17], [4,5,2,3,6,7,8]],
              123:[[10,11,15,11,12,13,14],[1,2,2,5,4,3,6]] , 
              124:[[14,11,10,13,16,16,17], [4,5,2,3,6,7,8]],
              125:[[10,11,15,11,12,13,14],[1,2,2,5,4,3,6]] , 
              126:[[14,11,10,13,16,16,17], [4,5,2,3,6,7,8]],
              128:[[10,11,15,11,12,13,14],[1,2,2,5,4,3,6]] , 
              129:[[14,11,10,13,16,16,17], [4,5,2,3,6,7,8]],
              51:[[10.422,11.452452,15.42452,16.4252,16.545,17.45243,17.45278],[1,2,2,5,7,6]] , 
              52:[[14,11,10,13,16,16,17], [4,5,2,3,6,7,8]],
              53:[[10,11,15,11,12,13,14],[1,2,2,5,4,3,6]] , 
              54:[[14,11,10,13,16,16,17], [4,5,2,3,6,7,8]],
              55:[[10,11,15,11,12,13,14],[1,2,2,5,4,3,6]] , 
              56:[[14,11,10,13,16,16,17], [4,5,2,3,6,7,8]],
              58:[[10,11,15,11,12,13,14],[1,2,2,5,4,3,6]] , 
              59:[[14,11,10,13,16,16,17], [4,5,2,3,6,7,8]],
              210:[[10,11,15,11,12,13,14],[1,2,2,5,4,3,6]] , 
              212:[[14,11,10,13,16,16,17], [4,5,2,3,6,7,8]],
              213:[[10,11,15,11,12,13,14],[1,2,2,5,4,3,6]] , 
              214:[[14,11,10,13,16,16,17], [4,5,2,3,6,7,8]],
              215:[[10,11,15,11,12,13,14],[1,2,2,5,4,3,6]] , 
              216:[[14,11,10,13,16,16,17], [4,5,2,3,6,7,8]],
              218:[[10,11,15,11,12,13,14],[1,2,2,5,4,3,6]] , 
              219:[[14,11,10,13,16,16,17], [4,5,2,3,6,7,8]],
              221:[[10,11,15,11,12,13,14],[1,2,2,5,4,3,6]] , 
              222:[[14,11,10,13,16,16,17], [4,5,2,3,6,7,8]],
              223:[[10,11,15,11,12,13,14],[1,2,2,5,4,3,6]] , 
              224:[[14,11,10,13,16,16,17], [4,5,2,3,6,7,8]],
              225:[[10,11,15,11,12,13,14],[1,2,2,5,4,3,6]] , 
              226:[[14,11,10,13,16,16,17], [4,5,2,3,6,7,8]],
              228:[[10,11,15,11,12,13,14],[1,2,2,5,4,3,6]] , 
              229:[[14,11,10,13,16,16,17], [4,5,2,3,6,7,8]],
              61:[[10.422,11.452452,15.42452,16.4252,16.545,17.45243,17.45278],[1,2,2,5,7,6]] , 
              62:[[14,11,10,13,16,16,17], [4,5,2,3,6,7,8]],
              63:[[10,11,15,11,12,13,14],[1,2,2,5,4,3,6]] , 
              64:[[14,11,10,13,16,16,17], [4,5,2,3,6,7,8]],
              65:[[10,11,15,11,12,13,14],[1,2,2,5,4,3,6]] , 
              66:[[14,11,10,13,16,16,17], [4,5,2,3,6,7,8]],
              68:[[10,11,15,11,12,13,14],[1,2,2,5,4,3,6]] , 
              69:[[14,11,10,13,16,16,17], [4,5,2,3,6,7,8]],
              310:[[10,11,15,11,12,13,14],[1,2,2,5,4,3,6]] , 
              312:[[14,11,10,13,16,16,17], [4,5,2,3,6,7,8]],
              313:[[10,11,15,11,12,13,14],[1,2,2,5,4,3,6]] , 
              314:[[14,11,10,13,16,16,17], [4,5,2,3,6,7,8]],
              315:[[10,11,15,11,12,13,14],[1,2,2,5,4,3,6]] , 
              316:[[14,11,10,13,16,16,17], [4,5,2,3,6,7,8]],
              318:[[10,11,15,11,12,13,14],[1,2,2,5,4,3,6]] , 
              319:[[14,11,10,13,16,16,17], [4,5,2,3,6,7,8]],
              321:[[10,11,15,11,12,13,14],[1,2,2,5,4,3,6]] , 
              322:[[14,11,10,13,16,16,17], [4,5,2,3,6,7,8]],
              323:[[10,11,15,11,12,13,14],[1,2,2,5,4,3,6]] , 
              324:[[14,11,10,13,16,16,17], [4,5,2,3,6,7,8]],
              325:[[10,11,15,11,12,13,14],[1,2,2,5,4,3,6]] , 
              326:[[14,11,10,13,16,16,17], [4,5,2,3,6,7,8]],
              328:[[10,11,15,11,12,13,14],[1,2,2,5,4,3,6]] , 
              329:[[14,11,10,13,16,16,17], [4,5,2,3,6,7,8]]}          


In [6]:
dd=MapToD(titrationObj)
widgets.interactive(dd.scatterPlot, residues=dd.selector, splitted=dd.split)

A Jupyter Widget